<a href="https://colab.research.google.com/github/rgukhui/cm4709/blob/main/Lab05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CM4709 Computer Vision
#Lab 05 Transfer Learning

##Aims
In this lab, you will learn how to:
1. Load image files into memory.
1. Pre-process images.
1. Use a CNN to classify a medical images.
1. Use Transfer Training for classification.

##Upload Images to GoogleDrive & Mount to Runtime
The dataset we use consists of images in sub-folders.
As files in a Colab runtime get deleted when the runtime is stopped, it is recommended that you put the dataset into your GoogleDrive.


1. Download the ZIP file from the link in Moodle.
1. Decompress the ZIP in your local machine before uploadint to GoogleDrive.
1. Once you get the folders and files, upload the whole folder structure to your GoogleDrive.


Once you mount your GoogleDrive to your Colab runtime, you will see the folders.

In [ ]:
#
#mount GoogleDrive to Colan runtime
#
#GoogleDrive will be at the path '/content/gdrive/MyDrive'.
#
from google.colab import drive
drive.mount('/content/gdrive')

##Define Common Parameters

Next, we define some parameters.

In [ ]:
import os

#path to dataset root folder
#***You may need to modify this.***
#
main_path='/content/gdrive/MyDrive/cm4709/Lab05/data'
train_folder_path=main_path+'/train'
test_folder_path=main_path+'/test'
val_folder_path=main_path+'/val'

#get class labels from subfolders under training data subset
labels = os.listdir(train_folder_path)

#image size
img_size = 112

#batch size in training
batch_size = 16

#no. of epochs
n_epochs = 25

#print sub-folder names
print('Class labels: ',labels)

##Explore Datasets
Get the count of instances in different folders.

In [ ]:
#print out image counts in train, test, val folders
#
train_n_path=train_folder_path+'/NORMAL'
train_p_path=train_folder_path+'/PNEUMONIA'

test_n_path=test_folder_path+'/NORMAL'
test_p_path=test_folder_path+'/PNEUMONIA'

val_n_path=val_folder_path+'/NORMAL'
val_p_path=val_folder_path+'/PNEUMONIA'

print('No. of -ve in training: ',len(os.listdir(train_n_path)))
print('No. of +ve in training: ',len(os.listdir(train_p_path)))

print('No. of -ve in testing: ',len(os.listdir(test_n_path)))
print('No. of +ve in testing: ',len(os.listdir(test_p_path)))

print('No. of -ve in validation: ',len(os.listdir(val_n_path)))
print('No. of +ve in validation: ',len(os.listdir(val_p_path)))


##Function to Load Image Files
Define some functions sample the files and to load image into memory.
As you may want to load a small set for testing, there is a `sample_rate` parameter for you to select how much from the original dataset to load.

Note that we only load in the file pathname for selection. An image file is only loaded if it is selected.

In [5]:
import numpy as np
import random
import cv2

#
#get random data sample in a folder with label
#folder: the main folder that contain the 2 classes of images
#labels: the 2 sub-folders under the main folder
#sample_rate: 0-1.0
#
#return a list of tuple (file_path,label) where label is 0/1 for normal/pnemonia
#labels: the subfolder names of NORMAL/PNEUMONIA
#folder: path to training/validation/testing sub-dataset
#rate: sampling rate of 0-1.0
#
def get_random_files(labels,dir,sample_rate):
  #list holding all filename and class label
  files_with_label=[]

  #go through both class labels of NORMAL/PNEUMONIA
  for label in labels:
    #append NORMAL or PNEUMONIA to give full path to images
    files=os.listdir(os.path.join(dir,label))

    #go through all files in folder
    for file in files:
      #get full pathname of file
      file_path=os.path.join(dir,label,file)

      #get class label as 0 or 1
      #0 is normal, 1 is pneumonia
      #
      class_num=labels.index(label)

      #append tuple to list
      files_with_label.append((file_path,class_num))

  #shuffle list of file names
  random.shuffle(files_with_label)

  #the no. of elements to pick from shuffled list
  number=int(len(files_with_label)*sample_rate)

  #pick out samples
  samples=random.sample(files_with_label,number)
  return samples

#
#load random images with label
#similar to get_random_data but actually loads in the images
#
#labels: the subfolders NORMAL/PNUEMONIA
#dir: path to training/validation/testing data subset
#img_size: a tuple in the format of (x,y) sizes/dimensions
#sample_rate: sampling rate in 0~1.0
#
def get_random_data(labels,dir,img_size,sample_rate):
  #
  #pick sample as a list of (file path,class label) tuples
  files_with_label=get_random_files(labels,dir,sample_rate)

  #lists to hold images and labels
  x=[]
  y=[]

  #go through all samples selected
  for file_with_label in files_with_label:
      try:
        #read image file
        (file,label)=file_with_label
        img=cv2.imread(file)

        #resize image
        resized_arr=cv2.resize(img,img_size)

        #append to both data and class lists
        x.append(resized_arr)
        y.append(label)
      except Exception as e:
        print(e)

  #convert x,y lists to numpy arrays before returning
  #also normalise images
  return (np.array(x)/255.0,np.array(y))


##Testing the Sampling Functions

We will test our get_random_files(...) and get_random_data(...) functions before we proceed. Also check out the results from both functions.

In [ ]:
#
#get 5% sample files from testing data
#
#get 10% sample of testing data, filenames only
sample_files=get_random_files(labels,test_folder_path,0.05)
print("No. of samples:",len(sample_files))
for file in sample_files:
  print(file)


In [ ]:
#
#get 1% sample files from testing data
#
#get 10% sample of testing data, filenames only
(data_x,data_y)=get_random_data(labels,test_folder_path,(100,100),0.01)
print("X shape: ",data_x.shape)
print("Y shape: ",data_y.shape)
print("1st image: ",data_x[0])
print("1st label: ",data_y[0])

##Load Datasets
We can now load our dataset into memory.
Because the original validation dataset is quite small, we won't use it.
Instead, we will split the training dataset into training and validation subsets.

In [8]:
#get the 2 sub-folders under training folder
#they are the class labels
#The following is executed in the beginning so we don't need to run it again.
#It is only repeated here as a reminder.
#
#labels=os.listdir(train_folder_path)

#only load 10% of dataset
sample_rate=0.10

#load full training set
(x_train_full,y_train_full)=get_random_data(labels,train_folder_path,(img_size,img_size),sample_rate)

#split full training set into train and validation sets
#10% of the full training set will be used for validation.
#
val_sample_rate=0.1
val_size=int(len(x_train_full)*val_sample_rate)
x_val=x_train_full[:val_size]   #validation set
y_val=y_train_full[:val_size]
x_train=x_train_full[val_size:] #training set
y_train=y_train_full[val_size:]

#load test set
(x_test,y_test)=get_random_data(labels,test_folder_path,(img_size,img_size),sample_rate)


##Examine Loaded Datasets
Show number of instances in each subset, and the shape of an instance.

In [ ]:
print('Dataset sizes')
print('=============')
print('x_train_full count: ',len(x_train_full))
print('y_train_full count: ',len(y_train_full))

print('x_train count: ',len(x_train))
print('y_train count: ',len(y_train))
print('x_val count: ',len(x_val))
print('y_val count: ',len(y_val))

#testing set
print('x_test count: ',len(x_test))
print('y_test count: ',len(y_test))

print('\nType & Shape')
print('============')
print('x_train type: ',type(x_train))
print('y_train type: ',type(y_train))
print('x_train data shape: ',x_train.shape)
print('y_train data shape: ',y_train.shape)
print('x_val type: ',type(x_val))
print('y_val type: ',type(y_val))
print('x_val data shape: ',x_val.shape)
print('y_val data shape: ',y_val.shape)
print('x_test type: ',type(x_test))
print('y_test type: ',type(y_test))
print('x_test data shape: ',x_test.shape)
print('y_test data shape: ',y_test.shape)

print('\nSamples')
print('=======')
print('x_train[0]: ',x_train[0])
print('y_train[0]: ',y_train[0])
#print('testing data shape: ',test_data.shape)

##Show Class Distribution

Class distribution in training dataset.

In [ ]:
import seaborn as sns

#
#a reusable function to show distribution of values in labels
#
#labels: an array of 0 or 1. 0s is NORMAL. 1 is PNEUMONIA.
#
def show_histogram(classes):
  l=[]  #array to store all class labels

  for c in classes:
      l.append(labels[c])   #use c as an index into the labels array

  #sns.set_style('whitegrid')
  sns.countplot(x=l)

#
#show distribution in training subset classes/labels
#
show_histogram(y_train)

Class distribution in validation dataset.

In [ ]:
#
#show distribution of classes/labels in validation subset
#
show_histogram(y_val)

Class validation in testing dataset.

In [ ]:
#
#show distribution of testing subset classes/labels
#
show_histogram(y_test)

##Show Random Image in Dataset
To make sure that the images and labels are correctly loaded into memory, we will show a random image from a chosen dataset, with its label.

In [ ]:
import matplotlib.pyplot as plt

#function to randomly show an image in a dataset
#
#x_data: an array of images
#y_data: an array of labels
#
def show_random_image(x_data,y_data):
  plt.figure(figsize=(3,3))

  #get a random index within the range of the array
  #
  index=random.randint(0,len(x_data)-1)

  #get image
  #
  image=x_data[index]

  #plot image
  plt.imshow(image)

  #get label and show it
  label=labels[y_data[index]]
  plt.title(label)

#
#show a random image in the testing dataset
#
show_random_image(x_test,y_test)

##Pre-process Images

Depending on the image quality, pre-processing the image may improve the classifier's performance.

In this example we will enhance the image contrast by using scikit-image library [`exposure.equalize_hist(...)` function](https://scikit-image.org/docs/stable/api/skimage.exposure.html#skimage.exposure.equalize_hist) to perform [histogram equalization](https://scikit-image.org/docs/stable/auto_examples/color_exposure/plot_equalize.html).

Note: There are many libraries and functions which can do this. You may find a similar function in OpenCV.

In [ ]:
#apply histogram equalisation

from skimage import exposure

#
#define function to perform histogram equalisation on an image
#
def histogram_equalise(image):
  image_eq=exposure.equalize_hist(image)
#  image_eq=cv2.equalizeHist(image)
  return image_eq


index=random.randint(0,len(x_train)-1)    #take a random image from the training dataset
before=x_train[index]             #original image
after=histogram_equalise(before)  #processed image
#
#show both images
#
plt.figure(figsize=(6,6))
plt.subplot(1,2,1)
plt.imshow(before)
plt.subplot(1,2,2)
plt.imshow(after)

##Build CNN Model
This CNN model is similar to what we did in the previous lab. There are, however, some extra layers added:
1. Some [`BatchNormalization` layer](https://keras.io/api/layers/normalization_layers/batch_normalization/) are added. This is often used to speed up the training process and make it more stable.
You can read more about [batch normalization here](https://towardsdatascience.com/batch-normalization-in-3-levels-of-understanding-14c2da90a338).
1. The [`Dropout` layer](https://keras.io/api/layers/regularization_layers/dropout/) is used to prevent overfitting of the model.
1. As we are doing binary classification, we have 1 single output neuron, with the Sigmoid activation function.

In [ ]:
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Activation,Dropout,AveragePooling2D,

#defined earlier
#just as a reminder
#
#img_size=112

# Model setup
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), strides = 1, padding = 'same', activation = 'relu', input_shape = (img_size, img_size,3)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2,2), strides = 2, padding = 'same'))
model.add(keras.layers.Conv2D(64, (3,3), strides = 1, padding = 'same', activation = 'relu', input_shape = (img_size, img_size,3)))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2,2), strides = 2, padding = 'same'))
model.add(keras.layers.Conv2D(64, (3,3), strides = 1, padding = 'same', activation = 'relu', input_shape = (img_size, img_size,3)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2,2), strides = 2, padding = 'same'))
model.add(keras.layers.Conv2D(128, (3,3), strides = 1, padding = 'same', activation = 'relu', input_shape = (img_size, img_size,3)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2,2), strides = 2, padding = 'same'))
model.add(keras.layers.Conv2D(256, (3,3), strides = 1, padding = 'same', activation = 'relu', input_shape = (img_size, img_size,3)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPool2D((2,2), strides = 2, padding = 'same'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units = 128, activation = 'relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(units = 1, activation = 'sigmoid'))

model.compile(
              optimizer = 'rmsprop', loss = 'binary_crossentropy',
              metrics = ['accuracy']
              )

model.summary()

##Just Train the Model
You can just go ahead and train the model if you wish.
The following code is similar to what we used in the last lab.

**However, if you want to tweak your training, wait and see the following cells.**

In [40]:
#train model
#
history=model.fit(x_train,y_train,epochs=n_epochs,batch_size=batch_size,
                  validation_data=(x_val,y_val))

Epoch 1/25
30/30 [==============================] - 1s 19ms/step - loss: 1.2961e-07 - accuracy: 1.0000 - val_loss: 0.1635 - val_accuracy: 0.9615
Epoch 2/25
30/30 [==============================] - 1s 18ms/step - loss: 6.7128e-07 - accuracy: 1.0000 - val_loss: 0.1133 - val_accuracy: 0.9615
Epoch 3/25
30/30 [==============================] - 1s 18ms/step - loss: 2.8368e-04 - accuracy: 1.0000 - val_loss: 0.3211 - val_accuracy: 0.9615
Epoch 4/25
30/30 [==============================] - 1s 17ms/step - loss: 3.9494e-08 - accuracy: 1.0000 - val_loss: 0.0496 - val_accuracy: 0.9808
Epoch 5/25
30/30 [==============================] - 1s 17ms/step - loss: 4.1471e-05 - accuracy: 1.0000 - val_loss: 0.0162 - val_accuracy: 0.9808
Epoch 6/25
30/30 [==============================] - 1s 18ms/step - loss: 1.3059e-07 - accuracy: 1.0000 - val_loss: 0.0067 - val_accuracy: 1.0000
Epoch 7/25
30/30 [==============================] - 1s 18ms/step - loss: 4.1917e-08 - accuracy: 1.0000 - val_loss: 0.0038 - val_ac

##Image Data Generator


Keras has a [`ImageDataGenerator`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) which can perform data augmentation.
We are just showing the code here.
Read the documentation for more details.

In [41]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

datagen = ImageDataGenerator(
  featurewise_center=False,
  samplewise_center=False,
  featurewise_std_normalization=False,
  samplewise_std_normalization=False,
  zca_whitening=False,
  rotation_range = 30,
  zoom_range = 0.2,
  width_shift_range = 0.1,
  height_shift_range = 0.1,
  horizontal_flip = True,
  vertical_flip=False)

datagen.fit(x_train)


##Cater for Class Imbalance
If you notice the distribution of classes in our dataset, there is a significant imbalance.
We can tell Keras to "pay more attention" to under-represented classes by passing a `class weight`.

The weight of each class is calculated by the following formulae:
<center>
$
weight_{0}=\frac{1}{neg} \times \frac{total}{2}
$

$
weight_{1}=\frac{1}{pos} \times \frac{total}{2}
$
</center>

See more information about **class weights** [here](https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights)


In [ ]:
#count of negative and positive cases
#
neg=0
pos=0

for y in y_train:
  if y==0:
    neg=neg+1
  else:
    pos=pos+1

#total number of instances
total=neg+pos

#calculate weights for both classes
#
weight_for_0 = 1 /neg * (total/2.0)
weight_for_1 = 1/pos *(total/2.0)

#put weights into a dictionary
#
class_weight = {0: weight_for_0, 1: weight_for_1}

print('Class weights: ',class_weight)

Class weights:  {0: 1.93801652892562, 1: 0.6738505747126436}


##Learning Rate

Sometime a model's training may get stuck. In this case reducing the learning rate usually helps. In Keras, you can add a callback function which reduces the learning rate if the performance metric does not improve after a few "patient" epochs. See the [ReduceLROnPlateau reference](https://keras.io/api/callbacks/reduce_lr_on_plateau/).

In [ ]:
from keras.callbacks import ReduceLROnPlateau

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 2, verbose = 1, factor = 0.3, min_lr = 0.000001)



In [ ]:

history = model.fit(datagen.flow(x_train, y_train, batch_size = batch_size),
                    epochs = n_epochs, validation_data = datagen.flow(x_val, y_val),
                    callbacks = [learning_rate_reduction], class_weight = class_weight)



##Show Training History

The following code to show the history is from the last lab.

In [ ]:
import pandas as pd
import matplotlib.pyplot as pyplot

#show keys in the history
print(history.history.keys())

#plot graphs
pd.DataFrame(history.history).plot(figsize=(15,8))
pyplot.grid(True)
pyplot.gca().set_ylim(0,1)
pyplot.show()

##Show Training Loss & Validation Loss

In [ ]:
losses = pd.DataFrame(model.history.history)
losses[['loss','val_loss']].plot()

##Show Training & Validation Accuracy

In [ ]:
losses[['accuracy','val_accuracy']].plot()

##Evaluate Using Testing Set

We can now use the testing dataset to evaluate our model.

In [79]:
#
#evaluate model over the whole testing dataset
#
(loss,accuracy)=model.evaluate(x_test,y_test)  #result is an array
#loss=result[0]      #1st value is loss
#accuracy=result[1]  #2nd value is accuracy

print("Loss of the model is - " ,loss)
print("Accuracy of the model is - " ,accuracy*100 , "%")

2/2 [==============================] - 0s 12ms/step - loss: 6.9016 - accuracy: 0.7581
Loss of the model is -  6.901576042175293
Accuracy of the model is -  75.80645084381104 %


##Predict 1 Image

In [ ]:
#from tensorflow.keras.models import load_model

index=random.randint(0,len(x_test)-1) #get a random index in testing subset
x_data=x_test[index]  #get image
y_data=y_test[index]  #get label

oneImage=np.expand_dims(x_data,axis=0)    #add 1 dimension to x
predictions=model.predict(oneImage)       #do prediction
pred_classes= np.where(predictions>0.5, 1, 0) #convert floating point number to integer
pred_label=labels[pred_classes[0][0]]     #get predicted label

plt.figure(figsize=(3,3))
plt.imshow(x_data)
real_label=labels[y_data]
plt.title("Real:"+real_label+" Predicted:"+pred_label)

##Precision, Recall, F1-score, and Support

We can get other performance metrics of the model.

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

#
#predict over the testing dataset
#
predictions=model.predict(x_test)
pred_labels= np.where(predictions>0.5, 1, 0)

#
#print report
#
print(classification_report(y_test, pred_labels, target_names = ['Pneumonia (Class 0)','Normal (Class 1)']))

##Show Confusion Matrix

In [ ]:
cf_matrix = confusion_matrix(y_test, pred_labels)
plt.figure(figsize = (10,10))
classes = ['Pneumonia','Normal']
labels = ['TN','FP','FN','TP']
labels = np.asarray(labels).reshape(2,2)

group_names = ['True Neg','False Pos','False Neg','True Pos']

group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap= "YlGn" ,
            xticklabels = classes,yticklabels = classes
           )

#Transfer Learning

We can use a pre-trained model to perform classification.
In this example, we will use [ResNet152V2](https://keras.io/api/applications/resnet/#resnet152v2-function).
You can explore other pre-trained models available in Keras.
You can find more information [here](https://keras.io/guides/transfer_learning/).

Note:
1. We freeze all layers in the pre-trained model.
1. We add a customed input layer, and some dense layers for the output.

In [ ]:
from keras.applications.vgg16 import VGG16 #(if you want to use VG16)
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

tf.keras.backend.clear_session()

base_model = tf.keras.applications.ResNet152V2(
    weights='imagenet',
    input_shape=(img_size, img_size, 3),
    include_top=False)

# freeze the layers
base_model.trainable = False

In [88]:
#function to get pre-trained model
#
def get_pretrained():

    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(img_size,img_size, 3))

    x = base_model(inputs)

    # Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.1)(x)

    #Final Layer (Output)
    output = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[inputs], outputs=output)

    return model


##Use Pre-trained Model Directly

In [ ]:
#
#get pre-trained model
#
model_pretrained = get_pretrained()

#
#compile model
#
model_pretrained.compile(loss='binary_crossentropy'
              , optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005), metrics='binary_accuracy')

#
#model summary
#
model_pretrained.summary()


##Fit Data into Pre-trained Model

While the layers from the pre-trained model are frozen, we still need to train the dense layers.

In [ ]:
"""
history_tl = model_pretrained.fit(datagen.flow(x_train,y_train, batch_size = batch_size) ,
                    epochs = n_epochs , validation_data = datagen.flow(x_val, y_val) ,
                    callbacks = [learning_rate_reduction],
                    steps_per_epoch = x_train.shape[0]/batch_size,
                    class_weight = class_weight
                   )
"""

history_tl=model_pretrained.fit(x_train,y_train,epochs=n_epochs,batch_size=batch_size,
                  validation_data=(x_val,y_val))

##Showing Training History of Pre-trained Model

In [ ]:
import pandas as pd
import matplotlib.pyplot as pyplot

#show keys in the history
print(history_tl.history.keys())

#plot graphs
pd.DataFrame(history_tl.history).plot(figsize=(15,8))
pyplot.grid(True)
pyplot.gca().set_ylim(0,1)
pyplot.show()

##Evaluate Pre-trained Model

In [ ]:
(loss,accuracy)=model_pretrained.evaluate(x_test,y_test)
print("Loss of the model is - " , loss)
print("Accuracy of the model is - " , accuracy*100 , "%")

##Fine-tuning Model

In the last example, we freeze all layers in the pre-trained ResNet model and only train the dense layers added by us.

To fine-tune the model, we can unfreeze some layers and retrain them specifically for this domain.
We usually keep the lower layers frozen as they capture generic patterns/features common in many domains.

In [93]:
#first set whole model trainable
#
base_model.trainable = True

#then freeze all except the last 10 layers
#
for layer in base_model.layers[:-10]:
    layer.trainable = False

##Compile Fine-tune Model

In [ ]:
model_pretrained.compile(loss='binary_crossentropy'
              , optimizer = tf.keras.optimizers.Adam(learning_rate=0.000002), metrics='binary_accuracy')

model_pretrained.summary()

##Fit Data Again
With the last 10 layers in the model made trainable, we will train it again.

In [ ]:
"""
history_ft = model_pretrained.fit(datagen.flow(x_train,y_train, batch_size = batch_size) ,
                    epochs = n_epochs , validation_data = datagen.flow(x_val, y_val) ,
                    callbacks = [learning_rate_reduction],
                    steps_per_epoch = x_train.shape[0]/batch_size,
                    class_weight = class_weight
                   )

"""
history_ft=model_pretrained.fit(x_train,y_train,epochs=30,
                  validation_data=(x_val,y_val))

##Show Training History

In [ ]:
import pandas as pd
import matplotlib.pyplot as pyplot

#show keys in the history
print(history.history.keys())

#plot graphs
pd.DataFrame(history.history).plot(figsize=(15,8))
pyplot.grid(True)
pyplot.gca().set_ylim(0,1)
pyplot.show()

##Evaluate Fine-tuned Model

In [ ]:
(loss,accuracy)=model_pretrained.evaluate(x_test,y_test)
print("Loss of the model is - " , loss)
print("Accuracy of the model is - " , accuracy*100 , "%")

In [ ]:
#
#predict over the testing dataset
#
predictions=model_pretrained.predict(x_test)
pred_labels= np.where(predictions>0.5, 1, 0)

#
#print report
#
print(classification_report(y_test, pred_labels, target_names = ['Pneumonia (Class 0)','Normal (Class 1)']))

##Confusion Matrix of Fine-tuned Model

In [ ]:
cf_matrix = confusion_matrix(y_test, pred_labels)
plt.figure(figsize = (10,10))

labels = ['TN','FP','FN','TP']
labels = np.asarray(labels).reshape(2,2)

group_names = ['True Neg','False Pos','False Neg','True Pos']

group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap= "YlGn",
           xticklabels = classes,yticklabels = classes)

##Clean Up
If you need to, the following code terminates the kernel and free memory before running new experiments.

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)
